In [ ]:
import warnings

warnings.filterwarnings('ignore')

## Load API tokens for our 3rd party APIs.

In [ ]:
from utils import get_circle_api_key, get_gh_api_key, get_google_api_key

cci_api_key = get_circle_api_key()
gh_api_key = get_gh_api_key()
vertex_api_key = get_google_api_key()

## Set up our github branch

In [ ]:
from utils import get_repo_name, get_branch

course_repo = get_repo_name()
course_repo
course_branch = get_branch()
course_branch

## The sample application: AI-powered quiz generator
We are going to build a AI powered quiz generator.
Create the dataset for the quizz.

In [ ]:
human_template = "{question}"
quiz_bank = """1. Subject: Leonardo DaVinci
   Categories: Art, Science
   Facts:
    - Painted the Mona Lisa
    - Studied zoology, anatomy, geology, optics
    - Designed a flying machine

2. Subject: Paris
   Categories: Art, Geography
   Facts:
    - Location of the Louvre, the museum where the Mona Lisa is displayed
    - Capital of France
    - Most populous city in France
    - Where Radium and Polonium were discovered by scientists Marie and Pierre Curie

3. Subject: Telescopes
   Category: Science
   Facts:
    - Device to observe different objects
    - The first refracting telescopes were invented in the Netherlands in the 17th Century
    - The James Webb space telescope is the largest telescope in space. It uses a gold-berillyum mirror

4. Subject: Starry Night
   Category: Art
   Facts:
    - Painted by Vincent van Gogh in 1889
    - Captures the east-facing view of van Gogh's room in Saint-Rémy-de-Provence

5. Subject: Physics
   Category: Science
   Facts:
    - The sun doesn't change color during sunset.
    - Water slows the speed of light
    - The Eiffel Tower in Paris is taller in the summer than the winter due to expansion of the metal."""

# Build the prompt template.
delimiter = "####"

prompt_template = f"""
Follow these steps to generate a customized quiz for the user.
The question will be delimited with four hashtags i.e {delimiter}

The user will provide a category that they want to create a quiz for. Any questions included in the quiz should only refer to the category.

Step 1:{delimiter} First identity the category user is asking about from the following list:
* Geography
* Science
* Art

Step 2:{delimiter} Determine the subjects to generate questions about. The list the topics are below:
{quiz_bank}

Pick up to two subjects that fit the user's category.

Step 3:{delimiter} Generate a quiz for the user. Based on the selected subjects generate 3 questions for the user using the facts about the subject.

Use the following format for the quiz:
Question 1:{delimiter} <question 1>

Question 2:{delimiter} <question 2>

Question 3:{delimiter} <question 3>
"""

In [ ]:
# Use langchain to build the prompt template.

from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([("human", prompt_template)])
# print to observe the content or generated object
chat_prompt

#Choose the LLM.
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model_name="gemini-pro",
                   convert_system_message_to_human=True)
llm
# Set up an output parser in LangChain that converts the llm response into a string.
# parser
from langchain.schema.output_parser import StrOutputParser

out_parser = StrOutputParser()
out_parser

# Connect the pieces using the pipe operator from Langchain Expression Language.
chain = chat_prompt | llm | out_parser
chain

Build the function 'assistance_chain' to put together all steps above.

In [ ]:
# taking all components and making reusable as one piece
def assistant_chain(
    system_message,
    human_template="{question}",
    llm=ChatVertexAI(model_name="gemini-pro",
                     convert_system_message_to_human=True),
    output_parser=StrOutputParser()):
  chat_prompt = ChatPromptTemplate.from_messages([
      ("system", system_message),
      ("human", human_template),
  ])
  return chat_prompt | llm | out_parser

### Evaluations
Create the function 'eval_expected_words' for the first example.

In [ ]:
def eval_expected_words(
    system_message,
    question,
    expected_words,
    human_template="{question}",
    llm=ChatVertexAI(model_name="gemini-pro",
                     convert_system_message_to_human=True),
    output_parser=StrOutputParser()):
  assistant = assistant_chain(system_message, human_template, llm,
                              output_parser)

  answer = assistant.invoke({"question": question})

  print(answer)

  assert any(word in answer.lower() for word in
             expected_words), f"Expected the assistant questions to include"

Test: Generate a quiz about science.

In [ ]:
question = "Generate a quiz about science."
expected_words = ["davinci", "telescope", "physics", "curie"]
# create the eval
eval_expected_words(prompt_template, question, expected_words)

Create the function 'evaluate_refusal' to define a failing test case where the app should decline to answer.

In [ ]:
def evaluate_refusal(
    system_message,
    question,
    decline_response,
    human_template="{question}",
    llm=ChatVertexAI(model_name="gemini-pro"),
    output_parser=StrOutputParser()):
  assistant = assistant_chain(human_template,
                              system_message,
                              llm,
                              output_parser)

  answer = assistant.invoke({"question": question})
  print(answer)

  assert decline_response.lower() in answer.lower(), \
    f"Expected the bot to decline with \
    '{decline_response}' got {answer}"

Define a new question (which should be a bad request)

In [ ]:
question = "Generate a quiz about Rome."
decline_response = "I'm sorry"

Create the refusal eval.
<p style="background-color:pink; padding:15px;"> <b>Note:</b> The following function call will throw an exception.</p>

In [ ]:
evaluate_refusal(prompt_template, question, decline_response)